---

title: [Shape-based interpolation](https://www.researchgate.net/publication/221400411_Efficient_Semiautomatic_Segmentation_of_3D_Objects_in_Medical_Images) (Schenk *et al*.)

author: Tiago Ribeiro

date: 04/05/2023

---

In [3]:
from watermark import watermark
print(watermark(author="\033[1m" + "Tiago Ribeiro"+ "\033[0m", 
                github_username="\033[1m" + "Tiago1Ribeiro"+ "\033[0m", 
                current_date=True, current_time=True, python=True, 
                updated=True, iversions=True, globals_= globals())
                )

Author: Tiago Ribeiro

Github username: Tiago1Ribeiro

Last updated: 2023-04-04 12:25:35

Python implementation: CPython
Python version       : 3.10.9
IPython version      : 8.11.0



### Setup

In [2]:
import os
import re
from shapely import wkt, multipolygons
from glob import glob

### Data sources & configurations

In [14]:
DATA_DIR = r"D:\BurnedAreaUAV_files\BurnedAreaUAV_dataset\BurnedAreaUAV_dataset"
# WKT containing the manually annotated polygons
WKT_FILE = os.path.join(DATA_DIR, 'WKT//train_valid.wkt')
# Directory to save PNG format annotated polygons
PNG_DIR = os.path.join(DATA_DIR, 'PNG//train_pngs')
# Directory to save PNG format interpolated polygons
# OUT_DIR = "E://BurnedAreaUAV_files//Interpolation//shape_interpol"
OUT_DIR = r"C:\Users\Utilizador\Desktop\output"
# Directories to save PNG format interpolated polygons
OUT_DIR_PNG = os.path.join(OUT_DIR, 'PNGs')
# Directory to save WKT format interpolated polygons
OUT_WKT_FILE = os.path.join(OUT_DIR, "shape_interpol.wkt")
# configs
ORIG_DIMS = (1280, 720)
OUT_DIMS = (1280, 720)

#### wkt2masc

In [10]:
def wkt2masc(wkt_file, images_path, orig_dims, out_dims, delete_files=True):
    """ 
    Converts WKT files to segmentation masks.
    Parameters:
        wkt_file {str} -- path to the WKT file
        images_path {str} -- path to the folder where the masks will be saved
        orig_dims {tuple} -- (width, height) original dimensions of the masks 
        out_dims {tuple} -- (width, height) output dimensions of the masks  
    Returns:
        Creates PNG images of the masks
    """

    os.makedirs(images_path, exist_ok=True)

    if delete_files:
        # delete files in the folder, if any
        for filename in os.listdir(images_path):
            if filename.endswith(".png"):
                os.remove(os.path.join(images_path, filename))

    # open WKT file
    wkt = open(wkt_file, 'r')
    num_lines = len(wkt.readlines())
    cnt = 0
    
    print(f"""
    {'-'*38}
    # \033[1mProperties of the resulting masks\033[0m
    # Width: {out_dims[0]}, Height: {out_dims[1]}
    # Number of masks to create: {num_lines}
    {'-'*38}
    """)
    
    # process each line of the WKT file
    with open(wkt_file) as f:
        for line in f:
            # extract numbers from the line
            points = [int(s) for s in re.findall('[0-9]+', line)]
            # create empty mask
            mask = np.zeros((orig_dims[1],orig_dims[0]), dtype=np.uint8)
            # create array with polygon points, with 2 columns (x,y)
            arr = np.array(points, dtype=np.int32).reshape((-1,2))
            # draw mask
            cv2.drawContours(image = mask,
                             contours=[arr],
                             contourIdx=-1,
                             color=(255, 255, 255),
                             thickness=-1,  # if > 0, thickness of the contour; if -1, fill object
                             lineType=cv2.LINE_AA)
            
            if out_dims != orig_dims:
                # resize frames with Lanczos interpolation
                mask = cv2.resize(mask, out_dims, interpolation=cv2.INTER_CUBIC)
            # save mask as PNG
            cv2.imwrite(os.path.join(images_path, f"frame_{cnt:06d}.png"), mask)
            cnt += 1
            # print progress
            print(f"\r\033[1m{cnt}\033[0m/{num_lines} masks created", end="\r")

### WKT to PNG Conversion

In [17]:
wkt2masc(WKT_FILE, OUT_DIR, ORIG_DIMS, OUT_DIMS, delete_files=False)


    --------------------------------------
    # Properties of the resulting masks
    # Width: 1280, Height: 720
    # Number of masks to create: 226
    --------------------------------------
    


### Shape Based Interpolation Function

#### interpolate_mask

In [5]:
import numpy as np
import cv2 
from scipy.interpolate import interp1d
from rasterio.features import rasterize

def interpolate_mask(polygons, start_frame, end_frame, int_instance, int_kind='linear', out_dim=None):
    """
    Interpolates between two segmentation masks using the distances of their respective contours.

    Args:
    - polygons (list): A list of polygon of shapely polygons for each frame in the video.
    - start_frame (int): The index of the first frame to interpolate between.
    - end_frame (int): The index of the second frame to interpolate between.
    - int_instance (float): The fraction of the distance between start_frame and end_frame to interpolate at.
    - int_kind (str): The kind of interpolation to use. Can be one of 'linear',
                     'nearest', 'zero', 'slinear','previous', or 'next'. Default is 'linear'.
    - out_dim (tuple): The desired output dimensions of the interpolated mask. Default is (720, 1280).

    Returns:
    - image (numpy array): The interpolated segmentation mask.
    """
    if not isinstance(polygons, list) or len(polygons) < 2:
        raise ValueError("polygons should be a list of binary masks for at least two frames")

    # Rasterize the binary masks for the start and end frames
    img1 = rasterize([polygons[start_frame]], out_shape=(720, 1280))
    img2 = rasterize([polygons[end_frame]], out_shape=(720, 1280))

    # Find the contours of the binary masks
    cnt1, _ = cv2.findContours(image=img1, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)
    cnt2, _ = cv2.findContours(image=img2, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)

    # Compute the distance transform of the masks
    mask1_in = np.zeros_like(img1)
    cv2.drawContours(image=mask1_in, contours=cnt1, contourIdx=-1, color=255, thickness=-1)
    mask1_out = cv2.bitwise_not(mask1_in)
    dist1_in = cv2.distanceTransform(mask1_in, distanceType=cv2.DIST_L2,
                                    maskSize=cv2.DIST_MASK_PRECISE, dstType=cv2.CV_32F)
    dist1_out = cv2.distanceTransform(mask1_out, distanceType=cv2.DIST_L2,
                                    maskSize=cv2.DIST_MASK_PRECISE, dstType=cv2.CV_32F)
    dist1 = dist1_in - dist1_out
    mask2_in = np.zeros_like(img2)
    cv2.drawContours(image=mask2_in, contours=cnt2, contourIdx=-1, color=255, thickness=-1)
    mask2_out = cv2.bitwise_not(mask2_in)
    dist2_in = cv2.distanceTransform(mask2_in, distanceType=cv2.DIST_L2,
                                    maskSize=cv2.DIST_MASK_PRECISE, dstType=cv2.CV_32F)
    dist2_out = cv2.distanceTransform(mask2_out, distanceType=cv2.DIST_L2,
                                    maskSize=cv2.DIST_MASK_PRECISE, dstType=cv2.CV_32F)
    dist2 = dist2_in - dist2_out
    
    # Interpolate the distance transforms using the specified interpolation method
    # int_instance_norm = (int_instance - start_frame) / (end_frame - start_frame)
    x_int = np.array(int_instance)
    x_samples = [0, 1]
    dist1_2d = np.reshape(dist1, (720*1280,))
    dist2_2d = np.reshape(dist2, (720*1280,))
    ifunc = interp1d(x_samples, np.stack((dist1_2d, dist2_2d), axis=1), axis=1, kind=int_kind)
    img_int_2d = ifunc(x_int)
    img_int = np.reshape(img_int_2d, (720, 1280, 1))

    image = np.where(img_int[:,:,0] > 0, 255, 0).astype(np.uint8)

    if out_dim is not None:
        image = cv2.resize(image, dsize= out_dim, interpolation=cv2.INTER_CUBIC)

    return image

In [6]:
def generate_interpolated_masks(polygons, out_dir, out_dim=None):
    """
    Generates interpolated masks for all frames in the video.

    Args:
    - polygons (list): A list of polygon of shapely polygons for each frame in the video.
    - out_dir (str): The path to the directory to save the interpolated masks to.
    - out_dim (tuple): (width, height)The desired output dimensions of the interpolated masks. 

    Returns:
    - None
    """
    if not isinstance(polygons, list) or len(polygons) < 2:
        raise ValueError("polygons should be a list of binary masks for at least two frames")

    # for each pair of frames, generate 99 interpolated masks
    cnt = 0
    for i in range(len(polygons) - 1):
        for j in range(1, 100):
            int_instance = i + j / 100
            image = interpolate_mask(polygons, i, i+1, int_instance, out_dim=out_dim)
            # jumps one number in frame count every 100 frames
            if j == 1:
                cnt += 1
            cv2.imwrite(os.path.join(out_dir, f"frame_{cnt:06}.png"), image)
            cnt += 1
            # print progress
            if j % 10 == 0:
                print(f"Generated frame_{cnt:06}.png", end="\r")
    print("\nDone.")

#### frames2video

In [18]:
def frames2video(img_list, nome_ficheiro='video', fps_ = 25, titulo: str = "", frame_num_text  = False, font_size: int = 1) -> None:
    """
    Converts a list of images into an AVI file with the same resolution as the first image in the list.
    Parameters: - list of PNG, TIFF, JPEG, BMP, WEBP, STK, LSM, or XCF images
    - name of the video file
    Returns: saves the video in the execution directory
    """

    # stores the dimensions of the first image
    img = cv2.imread(img_list[0])
    height, width, _ = img.shape
    size = (width, height)
    num_frames =  len(img_list)

    img_array = list()
    for i in range(len(img_list)):
        img = cv2.imread(img_list[i])
        img_array.append(img)
        print(f"1. Appending frames {i+1}/{num_frames}", end="\r")
        
    print("2. Creating video writer...", end="\r")
    video = cv2.VideoWriter(filename= nome_ficheiro + '.avi',
                            fourcc=cv2.VideoWriter_fourcc(*'mp4v'), fps = fps_,
                            frameSize=size)

    for i in range(len(img_array)):
        if frame_num_text:

            frame_number_text = f"frame_{i:06d}"
            cv2.putText(img_array[i], frame_number_text, (width-300, 50), 
                            cv2.FONT_HERSHEY_SIMPLEX,font_size, (255, 100, 100), 
                            2, cv2.LINE_AA)
        if titulo:
            cv2.putText(img_array[i], titulo, (50, 50), cv2.FONT_HERSHEY_SIMPLEX,
                        font_size, (255, 255, 255), 2, cv2.LINE_AA)
        
        video.write(img_array[i])
        print(f"3. Writing frames to file {i+1}/{num_frames}", end="\r")
    video.release()

In [19]:
generate_interpolated_masks(multipolygons, OUT_DIR, out_dim=(1280, 720))

ValueError: polygons should be a list of binary masks for at least two frames

In [111]:
frames2video(sorted(glob(os.path.join(OUT_DIR, "*.png"))), 
             nome_ficheiro='shape_based_interpol', fps_ = 25*10, 
             titulo="Shape Based Interpolation (10x speed)", 
             frame_num_text = True, font_size=1)

#### mask_to_polygons

In [20]:
from shapely.geometry import shape, MultiPolygon
from rasterio.features import shapes  
from rasterio import Affine        

def mask_to_polygons(mask_img):
    """
    Converts segmentation mask to shapely multipolygon.
    Adapted from: https://rocreguant.com/convert-a-mask-into-a-polygon-for-images-using-shapely-and-rasterio/1786/
    """
    all_polygons = list()
    
    for shp, _ in shapes(source=mask_img.astype(np.uint8),mask=(mask_img>0), 
                             transform=Affine(1.0, 0, 0, 0, 1.0, 0)):
        all_polygons.append(shape(shp))

    all_polygons = MultiPolygon(all_polygons)

    # Sometimes buffer() converts a simple Multipolygon to just a Polygon,
    # need to keep it a Multipolygon throughout
    if not all_polygons.is_valid:
        all_polygons = all_polygons.buffer(0)
        if all_polygons.type == 'Polygon':
            all_polygons = MultiPolygon([all_polygons])
    
    return all_polygons

#### msks_paths_to_polygon_list

In [21]:
def msks_paths_to_polygon_list(msks_paths):
    """
    Converts segmentation masks paths list to list of shapely multipolygons.
    """
    pol_list = list()
    for img_path in msks_paths:
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        polygon = mask_to_polygons(img)
        pol_list.append(polygon)
    return pol_list

In [ ]:
# generate list of shapely polygons from the masks
polygon_list = msks_paths_to_polygon_list(sorted(glob(os.path.join(OUT_DIR, "*.png"))))

# convert shapely list polygons to WKT format file
with open(OUT_WKT_FILE, 'w') as f:
    for polygon in polygon_list:
        f.write(polygon.wkt + '\n')

## Interpolation of the sampled polygons

#### Data Sources (sampled polygons)

In [23]:
OUT_DIR_SAMPLED_PNG = os.path.join(OUT_DIR, 'PNGs_sampled')
# create output directory
if not os.path.exists(OUT_DIR_SAMPLED_PNG):
    os.makedirs(OUT_DIR_SAMPLED_PNG)
# Directory to save WKT format interpolated polygons    
OUT_WKT_SAMPLED_FILE = os.path.join(OUT_DIR, "shape_interpol_sampled.wkt")
# WKT with sampled polygons
WKT_FILE_SAMPLED = os.path.join("E:/BurnedAreaUAV_files/Interpolation/reference_masks", "sampled_masks.txt")

In [25]:
WKT_FILE_SAMPLED = r"D:\BurnedAreaUAV_files\Interpolation\reference_masks\sampled_masks.txt"

# read txt file 
with open(WKT_FILE_SAMPLED, 'r') as f:
    polygons = f.readlines()
    # extract indexes and polygons
    indexes = [int(polygon.split(',')[0]) for polygon in polygons]
    polygons = [polygon.split(',', 1)[1][:-1] for polygon in polygons]
    # convert polygons to shapely polygons
    polygons = [wkt.loads(polygon) for polygon in polygons]

In [ ]:
cnt = 0
for i in range(len(polygons) - 1):
    for j in range(indexes[i]*100, indexes[i+1]*100):
        int_instance = i + j / 100
        image = interpolate_mask(polygons, i, i+1, int_instance, out_dim=OUT_DIMS)
        cv2.imwrite(os.path.join(OUT_DIR_SAMPLED_PNG, f"frame_{cnt:06}.png"), image)
        cnt += 1
        # print progress
        if j % 10 == 0:
            print(f"Generated frame_{cnt:06}.png", end="\r")
print("\nDone.")

In [18]:
cnt = 0
for i in range(len(polygons)- 1):
    for j in range(indexes[i]*100, indexes[i+1]*100):
        if j != indexes[i]*100 or j != indexes[i+1]*100:
            int_instance = (j-indexes[i]*100)/(indexes[i+1]*100 - indexes[i]*100)
            image = interpolate_mask(polygons, i, i+1, int_instance, out_dim=OUT_DIMS)
            cnt += 1
            cv2.imwrite(os.path.join(OUT_DIR_SAMPLED_PNG, f"frame_{cnt:06}.png"), image)
        else:
            cnt += 1
print("\nDone.")

Generated frame_022491.png
Done.


In [19]:
# convert sampled polygons to PNGs
for i, polygon in enumerate(polygons):
    image = np.zeros(OUT_DIMS[::-1], dtype=np.uint8)
    cv2.fillPoly(image, [np.array(polygon.exterior.coords).astype(np.int32)], 255)
    cv2.imwrite(os.path.join(OUT_DIR, f"frame_{(indexes[i]*100):06}.png"), image)
    print(f"Generated frame_{(indexes[i]*100):06}.png")

Generated frame_000000.png
Generated frame_001900.png
Generated frame_002800.png
Generated frame_004800.png
Generated frame_005600.png
Generated frame_007400.png
Generated frame_008200.png
Generated frame_008700.png
Generated frame_010600.png
Generated frame_012800.png
Generated frame_017300.png
Generated frame_019500.png
Generated frame_022500.png


In [20]:
frames2video(sorted(glob(os.path.join(OUT_DIR_SAMPLED_PNG, "*.png"))), 
             nome_ficheiro='shape_based_interpol_sampled', fps_ = 25*10, 
             titulo="Shape Based Interpolation - Sampled (10x speed)", 
             frame_num_text = True, font_size=1)

In [24]:
# generate list of shapely polygons from the masks
polygon_list = msks_paths_to_polygon_list(sorted(glob(os.path.join(OUT_DIR_SAMPLED_PNG, "*.png"))))

# convert shapely list polygons to WKT format file
with open(OUT_WKT_SAMPLED_FILE, 'w') as f:
    for polygon in polygon_list:
        f.write(polygon.wkt + '\n')